In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

t_path ="/content/drive/MyDrive/labeled_tomogram_data/tomograms"
label_path ="/content/drive/MyDrive/labeled_tomogram_data/labels"

tomo_dir = np.sort(os.listdir(t_path))
label_dir = np.sort(os.listdir(label_path))

#check if ordered
print(tomo_dir[0:5])
print(label_dir[0:5])

#Length of data
print(len(tomo_dir))
print(len(label_dir))

['PginABK10_001.h5.png' 'PginABK10_002.h5.png' 'PginABK10_003.h5.png'
 'PginABK10_004.h5.png' 'PginABK10_005.h5.png']
['PginABK10_001.txt' 'PginABK10_002.txt' 'PginABK10_003.txt'
 'PginABK10_004.txt' 'PginABK10_005.txt']
2233
2233


In [ ]:
#images to numpy array. This takes a second.
images = np.zeros((2233,1024,1024,1),dtype="int8")
for i,image in enumerate(tomo_dir):
  pic = Image.open(os.path.join(t_path,image))
  pix = np.array(pic)
  pix = np.resize(pix,(1024,1024,1))
  images[i] = pix

In [ ]:
#create list of 0s and 1s that corrispond to the "Good" and "Bad" labels. This is way too slow, can someone speed this up?
y=np.zeros(len(label_dir),dtype = "int8")
for i,f_name in enumerate(label_dir):
  with open(os.path.join(label_path,f_name), "r") as f:
    if f.readline()[0] == "G":
      y[i]=int(1)
  f.close()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, y, test_size=0.33, random_state=42)
del(images)
del(y)

In [ ]:
img_input = layers.Input(shape=(1024, 1024,1))

# First convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(64, 1, activation='relu')(img_input)
x = layers.MaxPooling2D(4)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 1, activation='relu')(x)
x = layers.MaxPooling2D(4)(x)

# Third convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 1, activation='relu')(x)
x = layers.MaxPooling2D(4)(x)

x = layers.Conv2D(8, 1, activation='relu')(x)
x = layers.MaxPooling2D(4)(x)

x = layers.Conv2D(4, 1, activation='relu')(x)
x = layers.MaxPooling2D(4)(x)

In [ ]:
# Flatten feature map to a 1-dim tensor so we can add fully connected layers
x = layers.Flatten()(x)

x = layers.Dense(4, activation='relu')(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(1, activation='sigmoid')(x)

# Create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully 
# connected layer + sigmoid output layer
model = Model(img_input, output)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024, 1024, 1)]   0         
                                                                 
 conv2d (Conv2D)             (None, 1024, 1024, 64)    128       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 256, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 32)      2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 16)        528   

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['acc'])

In [ ]:
history = model.fit(X_train,y_train,batch_size=7,epochs=10,validation_data=(X_test,y_test))

In [ ]:
# Retrieve a list of accuracy results on training and validation data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and validation data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')